<a href="https://colab.research.google.com/github/mykolesiko/eeg_investigation/blob/master/recognition_rnn_psv_many.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [8]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 100
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/lstm_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [33]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, input_size, hidden_size1, hidden_size2, koeff, len_seq):
       super().__init__()
       self.fc0 = nn.Linear(input_size, input_size)
       self.relu = nn.ReLU()
       self.rnn1 = nn.LSTM(input_size, hidden_size1, bidirectional = False, batch_first = True)
       self.drops = nn.ModuleList([nn.Dropout(koeff)] * len_seq)
       self.rnn2 = nn.LSTM(hidden_size1, hidden_size2, bidirectional = False, batch_first = True)
       input_linear_size = hidden_size2
       self.fc = nn.Linear(input_linear_size, 2)
   def forward(self, src):
      #src = [batch size, seq_len, input_size
      #src = self.fc0(src)
      #src = self.relu(src)
      #print(src.shape)
      encoder_outputs1, (hidden, c) = self.rnn1(src)  

      outputs1 = []
      for i, drop in enumerate(self.drops):
             temp = drop(encoder_outputs1[:, i, :]).unsqueeze(0)
             #print(temp.shape)
             outputs1.append(temp)
      outputs1 = torch.vstack(outputs1).permute(1, 0, 2)       
      #print(outputs1.shape)
      encoder_outputs2, (hidden, c) = self.rnn2(outputs1)  
      #print(hidden.shape)
      result = self.fc(hidden)
      #print(result.shape)
      return (result.squeeze(0))

In [10]:
INPUT_SIZE = 1 * 128 * 32
HIDDEN_SIZE_1 = 64
HIDDEN_SIZE_2 = 32
DROPOUT_COEFF = 0.5
LEN_SEQ = 10
def get_model(INPUT_SIZE, HIDDEN_SIZE_1, HIDDEN_SIZE_2, DROPOUT_COEFF, LEN_SEQ):
  model = EmotionNet(INPUT_SIZE, HIDDEN_SIZE_1, HIDDEN_SIZE_2, DROPOUT_COEFF, LEN_SEQ).to(device)
  return model


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model(INPUT_SIZE, HIDDEN_SIZE_1, HIDDEN_SIZE_2, DROPOUT_COEFF, LEN_SEQ)
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (fc0): Linear(in_features=4096, out_features=4096, bias=True)
  (relu): ReLU()
  (rnn1): LSTM(4096, 64, batch_first=True)
  (drops): ModuleList(
    (0): Dropout(p=0.5, inplace=False)
    (1): Dropout(p=0.5, inplace=False)
    (2): Dropout(p=0.5, inplace=False)
    (3): Dropout(p=0.5, inplace=False)
    (4): Dropout(p=0.5, inplace=False)
    (5): Dropout(p=0.5, inplace=False)
    (6): Dropout(p=0.5, inplace=False)
    (7): Dropout(p=0.5, inplace=False)
    (8): Dropout(p=0.5, inplace=False)
    (9): Dropout(p=0.5, inplace=False)
  )
  (rnn2): LSTM(64, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

In [46]:
import glob
import pickle
from collections import Counter



from collections import Counter
import scipy
from scipy import fft, ifft


# class RandomAugmentation(object):
#     def __init__(self, augmenters, probability):
#         self._augmenters = augmenters
#         self.probability = probability

#     def __call__(self, sample):
#         augmenter = random.choice(self._augmenters)
#         #print(augmenter)
#         if (augmenter):
#             if (np.random.random() < self.probability):
#                return augmenter(sample)
#         return(sample)  


# class add_noise(object):
#     def __init__(self, ):
#         super(add_noise, self).__init__()

#     def __call__(self, sample: dict):
#         white_noise = np.random.normal(loc=0.0, scale=1.0, size=sample['data'].shape)
#         sample['data'] = sample['data'] + white_noise
#         return sample


# class reset_part_in_time(object):
#     def __init__(self, percentage):
#         super(reset_part_in_time, self).__init__()
#         self.percentage = percentage

#     def __call__(self, sample: dict):
#         len_data = len(sample['data'][0])
#         interval_to_reset = int(len_data * self.percentage)
#         for ncanal in range(NVIDEOS):
#             begin = random.randint(0, len_data - interval_to_reset)
#             sample['data'][ncanal, begin : begin + interval_to_reset] = 0

#         return sample


# class reset_part_in_freq(object):
#     def __init__(self, percentage):
#         super(reset_part_in_freq, self).__init__()
#         self.percentage = percentage

#     def __call__(self, sample: dict):
#         len_data = len(sample['data'][0])
#         interval_to_reset = int(len_data * self.percentage)
#         power_freq_data = fft(sample['data'])
#         for ncanal in range(NVIDEOS):
#             begin = random.randint(0, len_data - interval_to_reset)
#             power_freq_data[ncanal, begin : begin + interval_to_reset] = 0
#         sample['data'] = ifft(power_freq_data)
#         return sample
NSUBJECTS = 32
NVIDEOS = 40
NCANALS = 32
       

# class flatten(object):
#     def __init__(self):
#         super(flatten, self).__init__()
        

#     def __call__(self, sample: dict):
#         input = sample['data'].copy()
#         len_seq = len(input)
#         segments = np.array(input).reshape(len_seq, -1)
#         sample['data'] = segments
#         #print(sample)
#         return sample     

class to_psd(object):
       def __init__(self):
           super(to_psd, self).__init__()

       def __call__(self, sample: dict):

         input = sample['data'].copy()
         len_seq = len(input)
         input_new = []
         for time in range(len_seq):
              features = []
              freq_data = input[time]
              freq_ranges_gc = np.array([0, 3, 4, 7, 8, 9.5, 10.5, 12, 8, 12, 13, 16, 17,20, 21, 29, 13, 29, 30, 50])
              koeff = 0.5
              freq_ranges_ind = ((freq_ranges_gc)/0.5).astype(int)
              freq_data_new = freq_data#np.sqrt(np.array(freq_data).real ** 2 + np.array(freq_data).imag ** 2)
              #freq_data_new = scipy.stats.zscore(freq_data_new)
              for i in range(NCANALS):
                  #pcc_matrix = np.zeros((27, 27))
                      for s in range(10):    
                           ind_begin = freq_ranges_ind[s * 2]
                           ind_end = freq_ranges_ind[s * 2 + 1]
                           features.append(freq_data_new[i, ind_begin : ind_end].sum())
                           for j in range(s):
                             features.append(features[s]/features[j])
                        
              input_new.append(features)
         #sample['data'] = scipy.stats.zscore(np.array(input_new))
         sample['data'] = (np.array(input_new))
         return(sample)



  
class ToTensor(object):

    def __init__(self, ):
        super(ToTensor, self).__init__()

    def __call__(self, sample: dict):
        return {"labels": torch.tensor(sample["labels"], dtype=torch.long),
                "data": torch.tensor(sample["data"], dtype=torch.float32),
                } 


class EmotionDataset(Dataset):
   def __init__ (self, data, labels, transforms, inds, interval = 1, shift = 1, len_seq = 10):
       self.data = data
       self.labels = labels
       self.transforms = transforms
       self.interval = interval
       self.len_seq = len_seq
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       self.indexes = []
       for sub in range(len(data)):
            for nvideo in inds[sub]:
               for nsec in range(3, LEN_RECORD + 3, shift):
                  if ((nsec + len_seq * interval) > LEN_RECORD + 3):
                    break
                  self.indexes.append((sub, nvideo, nsec))

            
   def __len__(self):
        result =  len(self.indexes)
        return result

   def __getitem__(self, item):         
       sample = {}   
       (sub, nvideo, nsec) = self.indexes[item]
       sample['labels'] = self.labels[sub][nvideo, :]
       sample['data'] = []
       nsec_it = nsec
       for i in range(self.len_seq):
          sample['data'].append(self.data[sub][nvideo, :32, nsec_it * 128 : (nsec_it + self.interval) * 128])
          nsec_it += self.interval
       
       if self.transforms is not None:
           for t in self.transforms:
                sample = t(sample)
       #print(sample)         
       return sample
       

In [13]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    labels.append(raw_data['labels'])
    #break

In [ ]:
# import glob
# import pickle
# from sklearn.preprocessing import MinMaxScaler

# data = []
# labels = []
# data_dir = './data_preprocessed_python'
# files = glob.glob(os.path.join(data_dir, "*.dat"))
# data_raw = []
# for file_data in files:
#     raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
#     data.append(raw_data['data'])
#     labels.append(raw_data['labels'])
    #break

In [ ]:
# import glob
# import pickle
# from sklearn.preprocessing import MinMaxScaler

# data = []
# labels = []
# data_dir = './data_preprocessed_python'
# files = glob.glob(os.path.join(data_dir, "*.npy"))
# data_raw = []
# for file_data in files:
#     raw_data = np.load(open(file_data, 'rb'))
#     data.append(raw_data)
    #labels.append(raw_data['labels'])
    #break

In [ ]:
# import glob
# import pickle
# from sklearn.preprocessing import MinMaxScaler

# data_orig = []
# labels = []
# data_dir = './data_preprocessed_python'
# files = glob.glob(os.path.join(data_dir, "*orig.npy"))
# data_raw = []
# for file_data in files:
#     raw_data = np.load(open(file_data, 'rb'))
#     data_orig.append(raw_data)
    #labels.append(raw_data['labels'])
    #break

In [ ]:
# from matplotlib import pyplot as plt
# for i in range(1):
#   fig, ax = plt.subplots(1, 1, figsize = (70, 5))
#   ax.plot(data_orig[0][i, 0, :])

In [ ]:
# from matplotlib import pyplot as plt
# for i in range(1):
#   fig, ax = plt.subplots(1, 1, figsize = (70, 5))
#   ax.plot(data[1][i, 0, :])

In [ ]:
# from matplotlib import pyplot as plt
# for i in range(40):
#   fig, ax = plt.subplots(1, 1, figsize = (70, 5))
#   ax.plot(data_prep[2][i, 0, :])
#   #plt.show()

In [ ]:
# def normalize_data1():
#   for sub in range(NSUBJECTS):
#     for nvideo in range(NVIDEOS):
#         for ncanal in range(NCANALS):
#              std = np.std(data[sub][nvideo, ncanal, :])
#              data[sub][nvideo, ncanal, :] = np.clip(data[sub][nvideo, ncanal, :], -3 * std, 3 * std)
#     scaler = MinMaxScaler(feature_range=(-1, 1))
#     data[sub][nvideo, :, :] = scaler.fit_transform(data[sub][nvideo,:, :].reshape(-1, 1)).reshape(1, 40, -1)         

# def normalize_data2():
#   for sub in range(NSUBJECTS):
#     for nvideo in range(NVIDEOS):
#         for ncanal in range(NCANALS):
#              std = np.std(data[sub][nvideo, ncanal, :])
#              data[sub][nvideo, ncanal, :] = np.clip(data[sub][nvideo, ncanal, :], -3 * std, 3 * std)
#              scaler = MinMaxScaler(feature_range=(-1, 1))
#              data[sub][nvideo, ncanal, :] = scaler.fit_transform(data[sub][nvideo, ncanal, :].reshape(-1, 1)).reshape(1, -1)


# def normalize_data3():
#   for sub in range(NSUBJECTS):
#     for nvideo in range(NVIDEOS):
#         for ncanal in range(NCANALS):
#              #std = np.std(data[sub][nvideo, ncanal, :])
#              #data[sub][nvideo, ncanal, :] = np.clip(data[sub][nvideo, ncanal, :], -3 * std, 3 * std)
#              scaler = MinMaxScaler(feature_range=(-1, 1))
#              data[sub][nvideo, ncanal, :] = scaler.fit_transform(data[sub][nvideo, ncanal, :].reshape(-1, 1)).reshape(1, -1) 
#              data[sub][nvideo, ncanal, :] = data[sub][nvideo, ncanal, :] - data[sub][nvideo, ncanal, :].mean()

# def normalize_data4():
#   for sub in range(len(data)):
#      scaler = MinMaxScaler(feature_range=(-1, 1))
#      data[sub][:, :, :] = scaler.fit_transform(data[sub][:, :, :].reshape(-1, 1)).reshape(40, 32, -1)             

# def zscore_data():
#   for sub in range(NSUBJECTS):
#     for nvideo in range(NVIDEOS):
#         for ncanal in range(NCANALS):
#              data[sub][nvideo, ncanal, :] = scipy.stats.zscore(data[sub][nvideo, ncanal, :])
#              #data[sub][nvideo, ncanal, :] = data[sub][nvideo, ncanal, :] - data[sub][nvideo, ncanal, :].mean()     

# normalize_data3()

In [15]:
type_emotion = 0

In [16]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score
k  = 5
labels_bin = []
for sub in range(32):
  temp = labels[sub] >= 4.5
  #print(labels[i])
  #print(temp)
  labels_bin.append(temp)
  print(sum(labels_bin[sub][:, type_emotion]), end=' ')




20 28 33 30 26 23 24 27 23 20 20 22 17 26 30 26 28 25 22 32 19 24 27 33 25 23 28 29 22 30 17 29 

In [ ]:
print(labels_bin[0][:, 0])

[ True  True  True  True  True  True  True  True False False False False
 False False False  True False  True  True  True  True  True  True  True
  True  True  True False False False False False False False False False
 False False False  True]


In [17]:
from sklearn.model_selection import StratifiedKFold, KFold 
from sklearn.metrics import f1_score, accuracy_score
k  = 5
labels_bin = []
for sub in range(32):
  temp = labels[sub] >= 4.5
  #print(labels[i])
  #print(temp)
  labels_bin.append(temp)
  print(sum(labels_bin[sub][:, type_emotion]), end=' ')

inds_train = []
inds_test = []
for sub in range(32):  
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = KFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    for ind_train, ind_test in  balanced_split:
        #print(ind_train, ind_test)
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        inds_train.append(ind_train)
        inds_test.append(ind_test)

20 28 33 30 26 23 24 27 23 20 20 22 17 26 30 26 28 25 22 32 19 24 27 33 25 23 28 29 22 30 17 29 

In [ ]:
# transforms = [to_segments(),ToTensor()] 
# train_dataset = EmotionDataset(data, labels_bin, transforms, inds_train)
# train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                                  pin_memory=True, shuffle=True, drop_last=True)



# val_dataset = EmotionDataset(data, labels_bin, transforms,  inds_test)
# val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                                pin_memory=True, shuffle=False, drop_last=False)

In [ ]:
# sample = val_dataset.__getitem__(7)
# print(sample['data'].shape)
# print(sample['data'])
# print(sample['labels'])

In [18]:
writer = None

In [19]:
def train(model, loader, criterion, optimizer, device, val_dataloader, val_f1_min,  description, batch = None, writer = None):
    model.train()
    train_loss = []
    inputs = []
    #torch.autograd.set_detect_anomaly(True)
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for i , batch in enumerate(loader):#, total=len(loader), desc="training...", position=0 , leave = True)):
            model.train()
            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
            #print(batch)
            #print(trg.shape)
            #print(src.shape)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
           
            #break
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            loss = criterion(levels_pred, trg) 

            # loss = 0
            # #loss_all = []
            
            # for class_value in range(2):
            #       mask = (trg == class_value) 
            #       levels_pred_sub = levels_pred[mask]
            #       trg_sub = trg[mask]
            #       #len += trg.shape[0]
            #       #print(f"{trg.shape[0]}", end = " ")
            #       if (trg_sub.shape[0] != 0):
            #             #loss_all.append(criterion(levels_pred_sub, trg_sub)/trg_sub.shape[0])
            #             loss += criterion(levels_pred_sub, trg_sub)/trg_sub.shape[0] 
            # #loss = loss_all[0] + loss_all[1] + (loss_all[0] - loss_all[1]) ** 2
            train_loss.append(loss.item())
            #print(train_loss[-1])
            loss.backward()
            #print(train_loss[-1])
            optimizer.step()
            if writer:
                writer.add_scalar(f'{description}/training loss per batch',
                                  loss.item(),
                                  i)

            # if (i % 100 == 0):
            #     acc, f1 = (calculate_predictions(model, val_dataloader))
            #     if (f1 > val_f1_min):
            #           val_f1_min      = f1
            #           torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val.tgz"))
            #break
    return np.mean(train_loss), val_f1_min


In [20]:
def evaluate(model, loader, criterion, device, writer, description):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(loader):#, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
            #print(src.shape)


            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()
            loss = criterion(levels_pred, trg) 
            epoch_loss += loss.item() 


            if writer:
                writer.add_scalar(f'{description}/val loss per batch',
                                  loss.item(),
                                  s)
        
    return epoch_loss / s

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

def calculate_predictions(model, loader, output = False):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(loader):#, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
         

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

            
        if output == True:    
            print(accuracy_score(real, pred)) 
            print(confusion_matrix(real, pred))  
            print(classification_report(real, pred))  
        f1 = ((f1_score(real, pred, 'binary', pos_label = 0))  + (f1_score(real, pred, 'binary', pos_label = 1)))/2
        return (accuracy_score(real, pred)) , f1
        #plt.hist(real)

In [35]:
def train_loop(description, n_epochs = 10):
    #args.epochs = 10
    #criterion =  fnn.mse_loss
    train_loss_min = 10000
    val_f1_min = -10000

    #batch = next(iter(train_dataloader))
    for epoch in range(n_epochs):
          #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
    
          train_loss, val_f1_min  = train(model, train_dataloader, criterion, optimizer ,device, val_dataloader, val_f1_min ,  description , None,  writer)
          #if epoch % 500 == 0:
          if writer:
                writer.add_scalar(f"{description}/training loss per epoch",
                                        train_loss,
                                        epoch)
          print(train_loss)

          if (train_loss < train_loss_min):
                 train_loss_min      = train_loss
                 torch.save({
                         'model_state_dict': model.state_dict(),
                         'optimizer_state_dict': optimizer.state_dict(),
                       },
                       os.path.join(args.output_dir, "train.tgz")
            )  

          val_loss = evaluate(model, val_dataloader, criterion, device,  writer, description )
          # #break
          print(val_loss)
          #if writer:
          #      writer.add_scalar(f"{description}/val loss per epoch",
          #                        val_loss,
          #                        epoch)
          acc, f1 = (calculate_predictions(model, val_dataloader, False))
          print(acc, f1)
          if (acc > val_f1_min):
                       val_f1_min      = acc
                       torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val_{description}.tgz")) 

In [28]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
type_emotion = 0

In [ ]:
# from sklearn.model_selection import StratifiedKFold, KFold 
# from sklearn.metrics import f1_score, accuracy_score
# k  = 5
# labels_bin = []
# for sub in range(32):
#   temp = labels[sub] >= 4.5
#   #print(labels[i])
#   #print(temp)
#   labels_bin.append(temp)
#   print(sum(labels_bin[sub][:, type_emotion]), end=' ')

# inds_train = []
# inds_test = []
# for sub in range(32):  
#     X = np.arange(40)
#     y = np.array(labels_bin[sub][:, type_emotion])
#     skf = KFold(n_splits=k, random_state=None, shuffle=True)
#     balanced_split = skf.split(X, y)
#     for ind_train, ind_test in  balanced_split:
#         print(ind_train, ind_test)
#         #количестов меток 1 в трейне и тесте
#         print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
#         print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
#         inds_train.append(ind_train)
#         inds_test.append(ind_test)

In [ ]:
print(len(labels_bin))

32


In [ ]:
# transforms = [to_segments(),ToTensor()] 
# train_dataset = EmotionDataset(data, labels_bin, transforms, inds_train)
# train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                                  pin_memory=True, shuffle=True, drop_last=True)



# val_dataset = EmotionDataset(data, labels_bin, transforms,  inds_test)
# val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                                pin_memory=True, shuffle=False, drop_last=False)

In [ ]:
  # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  # model = get_model()
  # model.apply(initialize_weights)
  # criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
  # #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
  # optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
  # train_loop('lstm_all_subject',  20)
  # description = 'lstm_all_subject'
  # model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
  # #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
  # model.load_state_dict(model_state['model_state_dict'])
  # acc, f1 = calculate_predictions(model, val_dataloader, True)
  # acc_sub.append(acc)
  # f1_sub.append(f1)
  # print(acc, f1)

In [ ]:
import pandas as pd
LEN_RECORD = 60
args.batch_size = 4
k = 4
#transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
#transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
transforms = [to_psd(),ToTensor()] 
#transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
#transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   
acc_all = []
f1_all = []
for sub in range(0, 1): 
    print(f"***sub {sub}****") 
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    acc_sub = []
    f1_sub = []
    for fold, (ind_train, ind_test) in  enumerate(balanced_split):
        #print(ind_train, ind_test)
        print(f"***fold {fold}****") 
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        #ind_train = [ 0,  3,  4,  5,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21,  23, 26, 27, 28, 30, 31, 32, 33, 34, 36, 37, 38, 39]
        #ind_test = [ 1,  2,  9, 11, 12, 22, 24, 25, 29, 35]
        inds_train = []
        inds_test = []
        inds_train.append(ind_train)
        inds_test.append(ind_test)
        print(inds_train)
        print(inds_test)
        print(labels_bin[sub][inds_test, 0])

        train_dataset = EmotionDataset(data[sub : sub + 1 ], labels_bin[sub  : sub + 1], transforms, inds_train, interval = 2, len_seq = 10)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                        pin_memory=True, shuffle=True, drop_last=True)

        val_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[ sub : sub + 1], transforms,  inds_test, interval = 2, len_seq = 10)
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                      pin_memory=True, shuffle=False, drop_last=False)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        LEN_SEQ = 10
        INPUT_SIZE = 1760 
        DROPOUT_COEFF = 0.2
        HIDDEN_SIZE_2 = 500
        HIDDEN_SIZE_1 = 500
        model = get_model(INPUT_SIZE, HIDDEN_SIZE_1, HIDDEN_SIZE_2, DROPOUT_COEFF, LEN_SEQ)
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.1)
        train_loop('lstm_subject1',  2)
        description = 'lstm_subject1'
        model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, val_dataloader, True)
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
    acc_all.append(acc_sub)    
    f1_all.append(f1_sub)  
    #f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4', 'f1_fold5'])
    #acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4', 'acc_fold5'])
    f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4'])
    acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4'])
    
    f1_data_1.to_csv("f1_data_1_lstm222.csv")
    acc_data_1.to_csv("acc_data_1_lstm222.csv")  

***sub 0****
***fold 0****
[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 18,
       19, 23, 25, 26, 27, 28, 30, 32, 34, 36, 37, 38, 39])]
[array([10, 17, 20, 21, 22, 24, 29, 31, 33, 35])]
[[False  True  True  True  True  True False False False False]]
0.4661395699271162
0.8367145152068606
0.4926829268292683 0.45333333333333337
0.21186552005926154
0.8963488197940237
0.5097560975609756 0.5087010176524243
0.5097560975609756
[[ 95 110]
 [ 91 114]]
              precision    recall  f1-score   support

           0       0.51      0.46      0.49       205
           1       0.51      0.56      0.53       205

    accuracy                           0.51       410
   macro avg       0.51      0.51      0.51       410
weighted avg       0.51      0.51      0.51       410

0.5097560975609756 0.5087010176524243
***fold 1****
[array([ 0,  2,  3,  4,  6,  7,  8, 10, 11, 13, 14, 15, 16, 17, 20, 21, 22,
       23, 24, 26, 27, 29, 31, 32, 33, 34, 35, 36, 37, 39])]
[array([ 1

In [ ]:

args.batch_size = 4
k = 3
#transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
#transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
transforms = [to_segments(),ToTensor()] 
#transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
#transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   
acc_all = []
f1_all = []
for sub in range(1, 2): 
    print(f"***sub {sub}****") 
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    acc_sub = []
    f1_sub = []
    for fold, (ind_train, ind_test) in  enumerate(balanced_split):
        #print(ind_train, ind_test)
        print(f"***fold {fold}****") 
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        inds_train = []
        inds_test = []
        inds_val = []

        ind_val = random.sample(list(ind_test), 2)
        ind_test = list(set(ind_test).difference(ind_val))
        print(ind_train, ind_test, ind_val)
        inds_train.append(ind_train)
        inds_test.append(ind_test)
        inds_val.append(ind_val)

        print(inds_train)
        print(inds_test)
        print(labels_bin[sub][inds_test, 0])

        train_dataset = EmotionDataset(data[sub : sub + 1 ], labels_bin[sub  : sub + 1], transforms, inds_train)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                        pin_memory=True, shuffle=True, drop_last=True)

        val_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[ sub : sub + 1], transforms,  inds_test)
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                      pin_memory=True, shuffle=False, drop_last=False)
        

        test_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[ sub : sub + 1], transforms,  inds_val)
        test_dataloader = DataLoader(test_dataset, batch_size=2, num_workers=1,
                                      pin_memory=True, shuffle=False, drop_last=False)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model()
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        train_loop('lstm_subject1',  20)
        description = 'lstm_subject1'
        model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, test_dataloader, True)
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
    acc_all.append(acc_sub)    
    f1_all.append(f1_sub)  
    #f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4', 'f1_fold5'])
    #acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4', 'acc_fold5'])
    f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4'])
    acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4'])
    
    f1_data_1.to_csv("f1_data_1_lstm222.csv")
    acc_data_1.to_csv("acc_data_1_lstm222.csv")  

In [ ]:
f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4'])
acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4'])
    
f1_data_1.to_csv("f1_data_1_lstm222.csv")
acc_data_1.to_csv("acc_data_1_lstm222.csv")  

In [ ]:
print(acc_data_1)
print(f1_data_1)

In [ ]:
print(labels_bin[1][[ 9, 12, 15, 16, 19, 25, 34, 35, 38, 39], 0])

In [ ]:
f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4'])
acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4'])
    
f1_data_1.to_csv("f1_data_1_lstm2.csv")
acc_data_1.to_csv("acc_data_1_lstm2.csv")  

In [ ]:
import pandas as pd
f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4', 'f1_fold5'])
acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4', 'acc_fold5'])
f1_data_1.to_csv("f1_data_1_lstm.csv")
acc_data_1.to_csv("acc_data_1_lstm.csv")

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
f1_data_1 = pd.read_csv("f1_data_1_lstm1.csv")
acc_data_1 = pd.read_csv("acc_data_1_lstm1.csv")


In [ ]:
print(acc_data_1[ ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4']]. mean(axis = 1).mean())
print(f1_data_1[ ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4']]. mean(axis = 1).mean())

In [ ]:
f1_data_1 = pd.to_csv("f1_data_1_lstm.csv")
acc_data_1.to_csv("acc_data_1_lstm.csv")

In [ ]:
description = 'lstm_subject1'
model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
#   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
model.load_state_dict(model_state['model_state_dict'])
acc, f1 = calculate_predictions(model, val_dataloader, True)
print(acc, f1)

In [ ]:
# for i , batch in enumerate(train_dataloader):#, total=len(loader), desc="training...", position=0 , leave = True)):
#             model = get_model()
#             model.train()
#             optimizer.zero_grad()
#             src  = batch['data'].to(device)
#             #print(src.shape)
#             trg = batch['labels'][:, 0]
#             #print(batch)
#             #print(trg.shape)
#             print(type(src))
#             print(src.shape)
#             levels_pred = model(src)  # B x (2 * NUM_PTS)
           
            break